### 开始

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
from tensorflow.keras import backend as K

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(devices=physical_devices[0], device_type='GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
keras.backend.set_image_data_format('channels_last')

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
#自定义matries 计算r2
def R2(y_true, y_pred):
    sst = K.sum(K.square(y_true - K.mean(y_true)))
    ssr = K.sum(K.square(y_pred - y_true))
    R2 = 1 - ssr/sst
    return R2

In [6]:
training_mode = 'model' # parameter
max_trials = 26
batch_size = 4096*4

### Unit Choices
units_choice = [[32,64,64,64],[32,64,64,128],[32,64,128,128],[32,128,128,128],[32,64,128,256]
                       [64,128,128,128],[64,128,128,256],[64,128,256,256],[64,256,256,256],[64,128,256,512],
                       [64,128,128,128,256],[64,128,128,256,256],[64,128,256,256,512]]

### Model

In [7]:
# Create the keras tuner model.
class MyHyperModel(keras_tuner.HyperModel):
    
    def build(self, hp):
        input_layer = layers.Input(shape = (18,1))
        out = input_layer
        units_choices = [[32,64,64,64],[32,64,64,128],[32,64,128,128],[32,128,128,128],[32,64,128,256],
                       [64,128,128,128],[64,128,128,256],[64,128,256,256],[64,256,256,256],[64,128,256,512],
                       [64,128,128,128,256],[64,128,128,256,256],[64,128,256,256,512]]
        choice = hp.Choice('units_choice',values = [i for i in range(len(units_choices))])
        for unit in units_choices[choice]:
            out = layers.Conv1D(filters=unit,kernel_size=hp.Choice('kernel_size',values = [3,5]),padding='same')(out)
            out = layers.BatchNormalization(momentum=0.8)(out)
            out = layers.Activation('relu')(out)

        out = layers.Flatten()(out)
        out = layers.BatchNormalization(momentum=0.8)(out)
        drop = hp.Boolean('dropout')
        out = layers.Dense(units=256,activation='relu')(out)
        if drop:
            out = layers.Dropout(0.2)(out)

        out = layers.Dense(units=256,activation='relu')(out)
        if drop:
            out = layers.Dropout(0.2)(out)

        out = layers.Dense(units=128,activation='relu')(out)
        if drop:
            out = layers.Dropout(0.2)(out) 

        final_out =  layers.Dense(1,activation=hp.Choice('activate_dense1',values = ['linear','sigmoid']))(out)
        model = keras.Model(inputs = input_layer, outputs = final_out)
        learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),loss='mse',metrics=[R2,'mae'])
        return model


### 2001

In [8]:
from sklearn.preprocessing import StandardScaler
year = 2001
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (426038, 18, 1)
X_validate shape: (127812, 18, 1)


In [9]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 10m 58s]
val_R2: 0.46270227432250977

Best val_R2 So Far: 0.8359763622283936
Total elapsed time: 02h 06m 06s
INFO:tensorflow:Oracle triggered exit


In [10]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00122, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2001_best.hdf5
27/27 - 8s - loss: 0.0011 - R2: 0.8535 - mae: 0.0244 - val_loss: 0.0012 - val_R2: 0.8361 - val_mae: 0.0260 - lr: 1.4993e-06 - 8s/epoch - 295ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.00122
27/27 - 6s - loss: 0.0011 - R2: 0.8482 - mae: 0.0244 - val_loss: 0.0012 - val_R2: 0.8325 - val_mae: 0.0262 - lr: 1.4993e-06 - 6s/epoch - 224ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00122
27/27 - 6s - loss: 0.0011 - R2: 0.8559 - mae: 0.0244 - val_loss: 0.0013 - val_R2: 0.8257 - val_mae: 0.0272 - lr: 1.4993e-06 - 6s/epoch - 212ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00122
27/27 - 6s - loss: 0.0011 - R2: 0.8548 - mae: 0.0244 - val_loss: 0.0012 - val_R2: 0.8352 - val_mae: 0.0262 - lr: 1.4993e-06 - 6s/epoch - 226ms/step
Epoch 5/300

Epoch 5: val_loss did not improve from 0.00122
27/27 - 6s - loss: 0.0011 - R2:

In [11]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2001
{'units_choice': 9, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.00014992878789921055}
1712/1712 [==============================] - 12s 7ms/step - loss: 0.0012 - R2: 0.8221 - mae: 0.0264


[0.0012353284982964396, 0.8220560550689697, 0.026356060057878494]

### 2002

In [12]:
from sklearn.preprocessing import StandardScaler
year = 2002
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (505950, 18, 1)
X_validate shape: (151785, 18, 1)


In [13]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 11m 58s]
val_R2: 0.7393009662628174

Best val_R2 So Far: 0.8670513033866882
Total elapsed time: 03h 31m 00s
INFO:tensorflow:Oracle triggered exit


In [14]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00093, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2002_best.hdf5
31/31 - 9s - loss: 6.5215e-04 - R2: 0.9066 - mae: 0.0186 - val_loss: 9.2507e-04 - val_R2: 0.8670 - val_mae: 0.0219 - lr: 1.2455e-06 - 9s/epoch - 290ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00093 to 0.00093, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2002_best.hdf5
31/31 - 7s - loss: 6.5252e-04 - R2: 0.9065 - mae: 0.0186 - val_loss: 9.2505e-04 - val_R2: 0.8671 - val_mae: 0.0219 - lr: 1.2455e-06 - 7s/epoch - 233ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00093 to 0.00092, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2002_best.hdf5
31/31 - 7s - loss: 6.5237e-04 - R2: 0.9065 - mae: 0.0186 - val_loss: 9.2489e-04 - val_R2: 0.8671 - val_mae: 0.0219 - lr: 1.2455e-06 - 7s/epoch - 221ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00092
31/31 - 7s - loss: 6.5286e-04 - R2: 0.9065 - mae: 0.0

In [15]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2002
{'units_choice': 11, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0012455335520847452}
2033/2033 [==============================] - 26s 13ms/step - loss: 9.3658e-04 - R2: 0.8576 - mae: 0.0220


[0.00093657715478912, 0.8575628399848938, 0.02198295295238495]

### 2003

In [16]:
from sklearn.preprocessing import StandardScaler
year = 2003
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (507932, 18, 1)
X_validate shape: (152380, 18, 1)


In [17]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 13m 04s]
val_R2: 0.7450388073921204

Best val_R2 So Far: 0.8559459447860718
Total elapsed time: 02h 53m 17s
INFO:tensorflow:Oracle triggered exit


In [18]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00101, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2003_best.hdf5
32/32 - 10s - loss: 9.8958e-04 - R2: 0.8549 - mae: 0.0233 - val_loss: 0.0010 - val_R2: 0.8526 - val_mae: 0.0238 - lr: 3.6573e-06 - 10s/epoch - 315ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00101 to 0.00100, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2003_best.hdf5
32/32 - 8s - loss: 9.8837e-04 - R2: 0.8560 - mae: 0.0233 - val_loss: 0.0010 - val_R2: 0.8545 - val_mae: 0.0237 - lr: 3.6573e-06 - 8s/epoch - 261ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00100
32/32 - 8s - loss: 9.8855e-04 - R2: 0.8545 - mae: 0.0233 - val_loss: 0.0010 - val_R2: 0.8544 - val_mae: 0.0235 - lr: 3.6573e-06 - 8s/epoch - 263ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00100
32/32 - 7s - loss: 9.8935e-04 - R2: 0.8582 - mae: 0.0233 - val_loss: 0.0010 - val_R2: 0.8535 - val_mae: 0.0238 - lr: 3.6573e-06 - 7s/epoch - 232

In [19]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2003
{'units_choice': 7, 'kernel_size': 5, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0036573086954592286}
2041/2041 [==============================] - 34s 16ms/step - loss: 0.0010 - R2: 0.8453 - mae: 0.0236


[0.0010202808771282434, 0.8453401923179626, 0.02358616515994072]

### 2004

In [20]:
from sklearn.preprocessing import StandardScaler
year = 2004
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (500833, 18, 1)
X_validate shape: (150250, 18, 1)


In [21]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 05m 37s]
val_R2: 0.6496430039405823

Best val_R2 So Far: 0.882961094379425
Total elapsed time: 04h 13m 43s
INFO:tensorflow:Oracle triggered exit


In [22]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00081, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2004_best.hdf5
31/31 - 10s - loss: 5.6481e-04 - R2: 0.9185 - mae: 0.0173 - val_loss: 8.0935e-04 - val_R2: 0.8829 - val_mae: 0.0204 - lr: 7.9152e-07 - 10s/epoch - 309ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.00081
31/31 - 7s - loss: 5.6429e-04 - R2: 0.9186 - mae: 0.0172 - val_loss: 8.0940e-04 - val_R2: 0.8829 - val_mae: 0.0204 - lr: 7.9152e-07 - 7s/epoch - 223ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00081 to 0.00081, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2004_best.hdf5
31/31 - 8s - loss: 5.6402e-04 - R2: 0.9187 - mae: 0.0172 - val_loss: 8.0903e-04 - val_R2: 0.8829 - val_mae: 0.0204 - lr: 7.9152e-07 - 8s/epoch - 244ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00081 to 0.00081, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2004_best.hdf5
31/31 - 6s - loss: 5.6412e-04 - R2: 0.9186 - mae: 0

In [23]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2004
{'units_choice': 10, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.000791524243562959}
2013/2013 [==============================] - 32s 16ms/step - loss: 7.8502e-04 - R2: 0.8780 - mae: 0.0203


[0.0007850177935324609, 0.8779571056365967, 0.020342692732810974]

### 2005

In [24]:
from sklearn.preprocessing import StandardScaler
year = 2005
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (505351, 18, 1)
X_validate shape: (151606, 18, 1)


In [25]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 07m 11s]
val_R2: 0.8803035616874695

Best val_R2 So Far: 0.883018970489502
Total elapsed time: 03h 45m 52s
INFO:tensorflow:Oracle triggered exit


In [26]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2005_best.hdf5
31/31 - 6s - loss: 6.8961e-04 - R2: 0.9023 - mae: 0.0192 - val_loss: 8.2082e-04 - val_R2: 0.8830 - val_mae: 0.0210 - lr: 2.5674e-06 - 6s/epoch - 192ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.00082
31/31 - 4s - loss: 6.9050e-04 - R2: 0.9022 - mae: 0.0192 - val_loss: 8.2090e-04 - val_R2: 0.8830 - val_mae: 0.0210 - lr: 2.5674e-06 - 4s/epoch - 134ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00082
31/31 - 4s - loss: 6.9015e-04 - R2: 0.9022 - mae: 0.0192 - val_loss: 8.2210e-04 - val_R2: 0.8828 - val_mae: 0.0210 - lr: 2.5674e-06 - 4s/epoch - 129ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00082 to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2005_best.hdf5
31/31 - 5s - loss: 6.9168e-04 - R2: 0.9020 - mae: 0.0193 - val_loss: 8.2077e-04 - val_R2: 0.8830 - val_mae: 0.0210 - lr: 2.5674e-06 - 

In [27]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2005
{'units_choice': 2, 'kernel_size': 5, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0025674422564504226}
2031/2031 [==============================] - 22s 11ms/step - loss: 8.4027e-04 - R2: 0.8734 - mae: 0.0211


[0.0008402664680033922, 0.8733550906181335, 0.02110433764755726]

### 2006

In [28]:
from sklearn.preprocessing import StandardScaler
year = 2006
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (513265, 18, 1)
X_validate shape: (153980, 18, 1)


In [29]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 06m 46s]
val_R2: 0.858832061290741

Best val_R2 So Far: 0.8645362854003906
Total elapsed time: 03h 46m 53s
INFO:tensorflow:Oracle triggered exit


In [30]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00091, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2006_best.hdf5
32/32 - 9s - loss: 6.2994e-04 - R2: 0.9063 - mae: 0.0183 - val_loss: 9.1059e-04 - val_R2: 0.8644 - val_mae: 0.0217 - lr: 6.7664e-07 - 9s/epoch - 280ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00091 to 0.00091, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2006_best.hdf5
32/32 - 7s - loss: 6.3016e-04 - R2: 0.9062 - mae: 0.0183 - val_loss: 9.0986e-04 - val_R2: 0.8645 - val_mae: 0.0217 - lr: 6.7664e-07 - 7s/epoch - 225ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00091
32/32 - 7s - loss: 6.3002e-04 - R2: 0.9065 - mae: 0.0183 - val_loss: 9.1034e-04 - val_R2: 0.8644 - val_mae: 0.0218 - lr: 6.7664e-07 - 7s/epoch - 211ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00091
32/32 - 7s - loss: 6.3014e-04 - R2: 0.9062 - mae: 0.0183 - val_loss: 9.0991e-04 - val_R2: 0.8645 - val_mae: 0.0218 - lr: 6.7664e-07 - 

In [31]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2006
{'units_choice': 10, 'kernel_size': 5, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0006766435736156065}
2063/2063 [==============================] - 28s 13ms/step - loss: 8.9253e-04 - R2: 0.8582 - mae: 0.0217


[0.0008925338042899966, 0.8582067489624023, 0.02168397791683674]

### 2007

In [32]:
from sklearn.preprocessing import StandardScaler
year = 2007
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (618216, 18, 1)
X_validate shape: (185465, 18, 1)


In [33]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 19m 19s]
val_R2: 0.8684059977531433

Best val_R2 So Far: 0.8888434767723083
Total elapsed time: 04h 21m 38s
INFO:tensorflow:Oracle triggered exit


In [34]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00085, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2007_best.hdf5
38/38 - 9s - loss: 6.7167e-04 - R2: 0.9119 - mae: 0.0191 - val_loss: 8.4713e-04 - val_R2: 0.8888 - val_mae: 0.0212 - lr: 2.5256e-06 - 9s/epoch - 226ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.00085
38/38 - 7s - loss: 6.7116e-04 - R2: 0.9120 - mae: 0.0190 - val_loss: 8.4739e-04 - val_R2: 0.8888 - val_mae: 0.0212 - lr: 2.5256e-06 - 7s/epoch - 182ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00085 to 0.00085, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2007_best.hdf5
38/38 - 7s - loss: 6.7205e-04 - R2: 0.9119 - mae: 0.0191 - val_loss: 8.4698e-04 - val_R2: 0.8889 - val_mae: 0.0212 - lr: 2.5256e-06 - 7s/epoch - 185ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00085
38/38 - 7s - loss: 6.7090e-04 - R2: 0.9120 - mae: 0.0190 - val_loss: 8.4722e-04 - val_R2: 0.8888 - val_mae: 0.0212 - lr: 2.5256e-06 - 

In [35]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2007
{'units_choice': 6, 'kernel_size': 5, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.002525622919996073}
2484/2484 [==============================] - 25s 10ms/step - loss: 8.4063e-04 - R2: 0.8825 - mae: 0.0211


[0.0008406255510635674, 0.8824700713157654, 0.021147213876247406]

### 2008

In [36]:
from sklearn.preprocessing import StandardScaler
year = 2008
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (625440, 18, 1)
X_validate shape: (187632, 18, 1)


In [37]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 11m 18s]
val_R2: 0.6717671751976013

Best val_R2 So Far: 0.9013871550559998
Total elapsed time: 04h 35m 55s
INFO:tensorflow:Oracle triggered exit


In [38]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00077, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2008_best.hdf5
39/39 - 8s - loss: 6.5625e-04 - R2: 0.9159 - mae: 0.0187 - val_loss: 7.7330e-04 - val_R2: 0.9008 - val_mae: 0.0203 - lr: 1.1664e-05 - 8s/epoch - 198ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.00077
39/39 - 5s - loss: 6.5737e-04 - R2: 0.9159 - mae: 0.0187 - val_loss: 7.7547e-04 - val_R2: 0.9005 - val_mae: 0.0203 - lr: 1.1664e-05 - 5s/epoch - 139ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00077
39/39 - 6s - loss: 6.5831e-04 - R2: 0.9153 - mae: 0.0187 - val_loss: 7.8805e-04 - val_R2: 0.8989 - val_mae: 0.0206 - lr: 1.1664e-05 - 6s/epoch - 143ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00077
39/39 - 6s - loss: 6.5691e-04 - R2: 0.9158 - mae: 0.0187 - val_loss: 7.9018e-04 - val_R2: 0.8986 - val_mae: 0.0206 - lr: 1.1664e-05 - 6s/epoch - 146ms/step
Epoch 5/300

Epoch 5: val_loss improved from 0.00077 to 0.0

In [39]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2008
{'units_choice': 4, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0011664403407834704}
2513/2513 [==============================] - 29s 12ms/step - loss: 7.9086e-04 - R2: 0.8912 - mae: 0.0207


[0.0007908611441962421, 0.8911523222923279, 0.020687131211161613]

### 2009

In [40]:
from sklearn.preprocessing import StandardScaler
year = 2009
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (613756, 18, 1)
X_validate shape: (184127, 18, 1)


In [41]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 08m 14s]
val_R2: 0.8871708512306213

Best val_R2 So Far: 0.8896893858909607
Total elapsed time: 04h 37m 08s
INFO:tensorflow:Oracle triggered exit


In [42]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2009_best.hdf5
38/38 - 10s - loss: 6.6525e-04 - R2: 0.9109 - mae: 0.0189 - val_loss: 8.2064e-04 - val_R2: 0.8896 - val_mae: 0.0209 - lr: 7.1342e-07 - 10s/epoch - 257ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.00082
38/38 - 8s - loss: 6.6497e-04 - R2: 0.9109 - mae: 0.0189 - val_loss: 8.2126e-04 - val_R2: 0.8895 - val_mae: 0.0209 - lr: 7.1342e-07 - 8s/epoch - 217ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00082 to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2009_best.hdf5
38/38 - 9s - loss: 6.6494e-04 - R2: 0.9110 - mae: 0.0189 - val_loss: 8.2051e-04 - val_R2: 0.8896 - val_mae: 0.0209 - lr: 7.1342e-07 - 9s/epoch - 233ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00082 to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2009_best.hdf5
38/38 - 9s - loss: 6.6521e-04 - R2: 0.9109 - mae: 0

In [43]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2009
{'units_choice': 8, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0007134180666249963}
2466/2466 [==============================] - 24s 10ms/step - loss: 8.1853e-04 - R2: 0.8833 - mae: 0.0209


[0.0008185265469364822, 0.8833301663398743, 0.02090991660952568]

### 2010

In [44]:
from sklearn.preprocessing import StandardScaler
year = 2010
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (650862, 18, 1)
X_validate shape: (195259, 18, 1)


In [45]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 02m 14s]
val_R2: -6.503274917602539

Best val_R2 So Far: 0.8944718241691589
Total elapsed time: 04h 03m 03s
INFO:tensorflow:Oracle triggered exit


In [46]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2010_best.hdf5
40/40 - 10s - loss: 6.2681e-04 - R2: 0.9195 - mae: 0.0183 - val_loss: 8.2327e-04 - val_R2: 0.8944 - val_mae: 0.0208 - lr: 7.1342e-07 - 10s/epoch - 262ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00082 to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2010_best.hdf5
40/40 - 9s - loss: 6.2617e-04 - R2: 0.9196 - mae: 0.0183 - val_loss: 8.2320e-04 - val_R2: 0.8944 - val_mae: 0.0208 - lr: 7.1342e-07 - 9s/epoch - 226ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00082
40/40 - 9s - loss: 6.2646e-04 - R2: 0.9195 - mae: 0.0183 - val_loss: 8.2323e-04 - val_R2: 0.8944 - val_mae: 0.0208 - lr: 7.1342e-07 - 9s/epoch - 215ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00082 to 0.00082, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2010_best.hdf5
40/40 - 9s - loss: 6.2840e-04 - R2: 0.9193 - mae: 0

In [47]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2010
{'units_choice': 8, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0007134180666249963}
2616/2616 [==============================] - 26s 10ms/step - loss: 8.0904e-04 - R2: 0.8879 - mae: 0.0208


[0.0008090377668850124, 0.8878769278526306, 0.02076665498316288]

### 2011

In [48]:
from sklearn.preprocessing import StandardScaler
year = 2011
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (657288, 18, 1)
X_validate shape: (197187, 18, 1)


In [49]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 01m 28s]
val_R2: -6.685684680938721

Best val_R2 So Far: 0.893876314163208
Total elapsed time: 03h 32m 43s
INFO:tensorflow:Oracle triggered exit


In [50]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00077, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2011_best.hdf5
41/41 - 6s - loss: 6.6714e-04 - R2: 0.9081 - mae: 0.0190 - val_loss: 7.7458e-04 - val_R2: 0.8939 - val_mae: 0.0205 - lr: 4.1282e-06 - 6s/epoch - 146ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.00077
41/41 - 4s - loss: 6.6692e-04 - R2: 0.9079 - mae: 0.0190 - val_loss: 7.7490e-04 - val_R2: 0.8938 - val_mae: 0.0204 - lr: 4.1282e-06 - 4s/epoch - 99ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00077
41/41 - 4s - loss: 6.6660e-04 - R2: 0.9083 - mae: 0.0190 - val_loss: 7.7468e-04 - val_R2: 0.8939 - val_mae: 0.0205 - lr: 4.1282e-06 - 4s/epoch - 104ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00077
41/41 - 4s - loss: 6.6657e-04 - R2: 0.9085 - mae: 0.0190 - val_loss: 7.7470e-04 - val_R2: 0.8938 - val_mae: 0.0205 - lr: 4.1282e-06 - 4s/epoch - 107ms/step
Epoch 5/300

Epoch 5: val_loss did not improve from 0.00077


In [51]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2011
{'units_choice': 2, 'kernel_size': 5, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.004128177547095777}
2641/2641 [==============================] - 24s 9ms/step - loss: 7.7608e-04 - R2: 0.8842 - mae: 0.0205


[0.000776084722019732, 0.8841922879219055, 0.02046854980289936]

### 2012 

In [52]:
from sklearn.preprocessing import StandardScaler
year = 2012
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (660527, 18, 1)
X_validate shape: (198158, 18, 1)


In [53]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 14m 02s]
val_R2: 0.9061621427536011

Best val_R2 So Far: 0.9061621427536011
Total elapsed time: 04h 17m 31s
INFO:tensorflow:Oracle triggered exit


In [54]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00076, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2012_best.hdf5
41/41 - 10s - loss: 6.3797e-04 - R2: 0.9214 - mae: 0.0185 - val_loss: 7.5872e-04 - val_R2: 0.9061 - val_mae: 0.0199 - lr: 2.5696e-06 - 10s/epoch - 247ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00076 to 0.00076, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2012_best.hdf5
41/41 - 9s - loss: 6.3824e-04 - R2: 0.9212 - mae: 0.0185 - val_loss: 7.5832e-04 - val_R2: 0.9061 - val_mae: 0.0198 - lr: 2.5696e-06 - 9s/epoch - 218ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00076
41/41 - 9s - loss: 6.3803e-04 - R2: 0.9214 - mae: 0.0185 - val_loss: 7.5928e-04 - val_R2: 0.9060 - val_mae: 0.0198 - lr: 2.5696e-06 - 9s/epoch - 216ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00076
41/41 - 9s - loss: 6.3871e-04 - R2: 0.9212 - mae: 0.0185 - val_loss: 7.6171e-04 - val_R2: 0.9057 - val_mae: 0.0199 - lr: 2.5696e-06 

In [55]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2012
{'units_choice': 10, 'kernel_size': 5, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0025696038852286752}
2654/2654 [==============================] - 30s 11ms/step - loss: 7.5580e-04 - R2: 0.9019 - mae: 0.0199


[0.0007557958015240729, 0.9018898606300354, 0.01985171064734459]

### 2013

In [56]:
from sklearn.preprocessing import StandardScaler
year = 2013
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (647136, 18, 1)
X_validate shape: (194140, 18, 1)


In [57]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 09m 57s]
val_R2: 0.819561779499054

Best val_R2 So Far: 0.9089248180389404
Total elapsed time: 05h 03m 07s
INFO:tensorflow:Oracle triggered exit


In [58]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00071, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2013_best.hdf5
40/40 - 12s - loss: 5.5070e-04 - R2: 0.9290 - mae: 0.0172 - val_loss: 7.0823e-04 - val_R2: 0.9089 - val_mae: 0.0194 - lr: 8.6725e-07 - 12s/epoch - 312ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00071 to 0.00071, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2013_best.hdf5
40/40 - 12s - loss: 5.5041e-04 - R2: 0.9290 - mae: 0.0172 - val_loss: 7.0776e-04 - val_R2: 0.9089 - val_mae: 0.0193 - lr: 8.6725e-07 - 12s/epoch - 288ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00071 to 0.00071, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2013_best.hdf5
40/40 - 12s - loss: 5.5017e-04 - R2: 0.9290 - mae: 0.0172 - val_loss: 7.0760e-04 - val_R2: 0.9089 - val_mae: 0.0193 - lr: 8.6725e-07 - 12s/epoch - 294ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00071
40/40 - 11s - loss: 5.5056e-04 - R2: 0.9289 - m

In [59]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2013
{'units_choice': 9, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0008672502254279293}
2601/2601 [==============================] - 26s 10ms/step - loss: 7.1029e-04 - R2: 0.9017 - mae: 0.0193


[0.0007102890522219241, 0.901692807674408, 0.019311318174004555]

### 2014

In [60]:
from sklearn.preprocessing import StandardScaler
year = 2014
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (603515, 18, 1)
X_validate shape: (181055, 18, 1)


In [61]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 13m 19s]
val_R2: 0.8121931552886963

Best val_R2 So Far: 0.9153472781181335
Total elapsed time: 03h 26m 35s
INFO:tensorflow:Oracle triggered exit


In [62]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00070, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2014_best.hdf5
37/37 - 8s - loss: 6.2181e-04 - R2: 0.9251 - mae: 0.0182 - val_loss: 7.0355e-04 - val_R2: 0.9152 - val_mae: 0.0193 - lr: 1.8756e-06 - 8s/epoch - 223ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.00070
37/37 - 7s - loss: 6.2069e-04 - R2: 0.9252 - mae: 0.0182 - val_loss: 7.0464e-04 - val_R2: 0.9151 - val_mae: 0.0194 - lr: 1.8756e-06 - 7s/epoch - 180ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00070
37/37 - 7s - loss: 6.2173e-04 - R2: 0.9251 - mae: 0.0182 - val_loss: 7.0499e-04 - val_R2: 0.9151 - val_mae: 0.0194 - lr: 1.8756e-06 - 7s/epoch - 178ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00070
37/37 - 7s - loss: 6.2209e-04 - R2: 0.9250 - mae: 0.0182 - val_loss: 7.1129e-04 - val_R2: 0.9143 - val_mae: 0.0195 - lr: 1.8756e-06 - 7s/epoch - 182ms/step
Epoch 5/300

Epoch 5: val_loss improved from 0.00070 to 0.0

In [63]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2014
{'units_choice': 7, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0018756180813858188}
2425/2425 [==============================] - 24s 10ms/step - loss: 6.9446e-04 - R2: 0.9096 - mae: 0.0193


[0.0006944561027921736, 0.9096441864967346, 0.019325878471136093]

### 2015

In [64]:
from sklearn.preprocessing import StandardScaler
year = 2015
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (606046, 18, 1)
X_validate shape: (181813, 18, 1)


In [65]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 17m 35s]
val_R2: 0.6960456371307373

Best val_R2 So Far: 0.904409646987915
Total elapsed time: 04h 15m 07s
INFO:tensorflow:Oracle triggered exit


In [66]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00076, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2015_best.hdf5
37/37 - 8s - loss: 5.9551e-04 - R2: 0.9264 - mae: 0.0180 - val_loss: 7.6073e-04 - val_R2: 0.9044 - val_mae: 0.0202 - lr: 6.3466e-07 - 8s/epoch - 221ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00076 to 0.00076, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2015_best.hdf5
37/37 - 7s - loss: 5.9556e-04 - R2: 0.9263 - mae: 0.0180 - val_loss: 7.6062e-04 - val_R2: 0.9044 - val_mae: 0.0202 - lr: 6.3466e-07 - 7s/epoch - 181ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00076
37/37 - 6s - loss: 5.9567e-04 - R2: 0.9263 - mae: 0.0180 - val_loss: 7.6079e-04 - val_R2: 0.9044 - val_mae: 0.0202 - lr: 6.3466e-07 - 6s/epoch - 158ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00076
37/37 - 7s - loss: 5.9551e-04 - R2: 0.9264 - mae: 0.0180 - val_loss: 7.6099e-04 - val_R2: 0.9044 - val_mae: 0.0202 - lr: 6.3466e-07 - 

In [67]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2015
{'units_choice': 6, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0006346630548574778}
2436/2436 [==============================] - 21s 8ms/step - loss: 7.6253e-04 - R2: -inf - mae: 0.0202


[0.0007625287398695946, -inf, 0.020168164744973183]

### 2016

In [68]:
from sklearn.preprocessing import StandardScaler
year = 2016
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (636232, 18, 1)
X_validate shape: (190870, 18, 1)


In [69]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 03m 13s]
val_R2: 0.821239173412323

Best val_R2 So Far: 0.924795925617218
Total elapsed time: 03h 42m 35s
INFO:tensorflow:Oracle triggered exit


In [70]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00064, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2016_best.hdf5
39/39 - 6s - loss: 4.9229e-04 - R2: 0.9427 - mae: 0.0162 - val_loss: 6.4468e-04 - val_R2: 0.9248 - val_mae: 0.0184 - lr: 3.0609e-06 - 6s/epoch - 148ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00064 to 0.00064, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2016_best.hdf5
39/39 - 4s - loss: 4.9214e-04 - R2: 0.9427 - mae: 0.0162 - val_loss: 6.4462e-04 - val_R2: 0.9248 - val_mae: 0.0184 - lr: 3.0609e-06 - 4s/epoch - 95ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00064 to 0.00064, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2016_best.hdf5
39/39 - 4s - loss: 4.9208e-04 - R2: 0.9427 - mae: 0.0162 - val_loss: 6.4455e-04 - val_R2: 0.9248 - val_mae: 0.0184 - lr: 3.0609e-06 - 4s/epoch - 114ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00064 to 0.00064, saving model to D:/SGYL/SM_results_data/check_poi

In [71]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2016
{'units_choice': 1, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0030608790965779946}
2557/2557 [==============================] - 24s 9ms/step - loss: 6.2884e-04 - R2: 0.9210 - mae: 0.0182


[0.000628840527497232, 0.9209580421447754, 0.018243618309497833]

### 2017

In [72]:
from sklearn.preprocessing import StandardScaler
year = 2017
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (637490, 18, 1)
X_validate shape: (191247, 18, 1)


In [73]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 03m 02s]
val_R2: -5.680734157562256

Best val_R2 So Far: 0.9211273789405823
Total elapsed time: 04h 01m 20s
INFO:tensorflow:Oracle triggered exit


In [74]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00068, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2017_best.hdf5
39/39 - 8s - loss: 5.5133e-04 - R2: 0.9361 - mae: 0.0172 - val_loss: 6.8142e-04 - val_R2: 0.9211 - val_mae: 0.0190 - lr: 1.9037e-06 - 8s/epoch - 198ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00068 to 0.00068, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2017_best.hdf5
39/39 - 6s - loss: 5.5169e-04 - R2: 0.9360 - mae: 0.0172 - val_loss: 6.8127e-04 - val_R2: 0.9211 - val_mae: 0.0190 - lr: 1.9037e-06 - 6s/epoch - 164ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00068 to 0.00068, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2017_best.hdf5
39/39 - 7s - loss: 5.5130e-04 - R2: 0.9361 - mae: 0.0172 - val_loss: 6.8089e-04 - val_R2: 0.9211 - val_mae: 0.0190 - lr: 1.9037e-06 - 7s/epoch - 169ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00068
39/39 - 7s - loss: 5.5112e-04 - R2: 0.9361 - mae: 0.0

In [75]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2017
{'units_choice': 6, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0019036600889013378}
2562/2562 [==============================] - 24s 10ms/step - loss: 6.8559e-04 - R2: 0.9145 - mae: 0.0191


[0.0006855916581116617, 0.9144935011863708, 0.01906181126832962]

### 2018

In [76]:
from sklearn.preprocessing import StandardScaler
year = 2018
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (614272, 18, 1)
X_validate shape: (184282, 18, 1)


In [77]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 15m 06s]
val_R2: 0.7567874789237976

Best val_R2 So Far: 0.9170046448707581
Total elapsed time: 04h 38m 46s
INFO:tensorflow:Oracle triggered exit


In [78]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00073, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2018_best.hdf5
38/38 - 7s - loss: 5.7402e-04 - R2: 0.9349 - mae: 0.0176 - val_loss: 7.2965e-04 - val_R2: 0.9170 - val_mae: 0.0197 - lr: 7.9689e-07 - 7s/epoch - 181ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.00073
38/38 - 5s - loss: 5.7448e-04 - R2: 0.9349 - mae: 0.0176 - val_loss: 7.2998e-04 - val_R2: 0.9170 - val_mae: 0.0197 - lr: 7.9689e-07 - 5s/epoch - 137ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00073 to 0.00073, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2018_best.hdf5
38/38 - 6s - loss: 5.7400e-04 - R2: 0.9349 - mae: 0.0176 - val_loss: 7.2961e-04 - val_R2: 0.9170 - val_mae: 0.0197 - lr: 7.9689e-07 - 6s/epoch - 147ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00073
38/38 - 5s - loss: 5.7383e-04 - R2: 0.9350 - mae: 0.0176 - val_loss: 7.2999e-04 - val_R2: 0.9170 - val_mae: 0.0197 - lr: 7.9689e-07 - 

In [79]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2018
{'units_choice': 7, 'kernel_size': 5, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.0007968852418666189}
2469/2469 [==============================] - 11s 4ms/step - loss: 7.3439e-04 - R2: 0.9101 - mae: 0.0198


[0.0007343881297856569, 0.9101068377494812, 0.01982777565717697]

### 2019

In [80]:
from sklearn.preprocessing import StandardScaler
year = 2019
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (625580, 18, 1)
X_validate shape: (187674, 18, 1)


In [81]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 14m 55s]
val_R2: 0.6352357268333435

Best val_R2 So Far: 0.9317988753318787
Total elapsed time: 02h 28m 52s
INFO:tensorflow:Oracle triggered exit


In [82]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00057, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2019_best.hdf5
39/39 - 9s - loss: 4.5610e-04 - R2: 0.9455 - mae: 0.0156 - val_loss: 5.6789e-04 - val_R2: 0.9318 - val_mae: 0.0173 - lr: 1.9256e-06 - 9s/epoch - 226ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.00057
39/39 - 7s - loss: 4.5648e-04 - R2: 0.9455 - mae: 0.0157 - val_loss: 5.6805e-04 - val_R2: 0.9318 - val_mae: 0.0173 - lr: 1.9256e-06 - 7s/epoch - 183ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00057
39/39 - 7s - loss: 4.5645e-04 - R2: 0.9452 - mae: 0.0156 - val_loss: 5.6909e-04 - val_R2: 0.9317 - val_mae: 0.0173 - lr: 1.9256e-06 - 7s/epoch - 183ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00057
39/39 - 7s - loss: 4.5663e-04 - R2: 0.9454 - mae: 0.0156 - val_loss: 5.6852e-04 - val_R2: 0.9318 - val_mae: 0.0173 - lr: 1.9256e-06 - 7s/epoch - 183ms/step
Epoch 5/300

Epoch 5: val_loss did not improve from 0.00057

In [83]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2019
{'units_choice': 9, 'kernel_size': 3, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.001925564703335844}
2514/2514 [==============================] - 11s 4ms/step - loss: 5.6996e-04 - R2: 0.9263 - mae: 0.0174


[0.0005699647590517998, 0.9263062477111816, 0.017356805503368378]

### 2020

In [84]:
from sklearn.preprocessing import StandardScaler
year = 2020
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (709183, 18, 1)
X_validate shape: (212755, 18, 1)


In [85]:

def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/CNN/',
            project_name=('CNN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 26 Complete [00h 09m 39s]
val_R2: 0.8425613641738892

Best val_R2 So Far: 0.9292113184928894
Total elapsed time: 03h 01m 12s
INFO:tensorflow:Oracle triggered exit


In [86]:
filepath="CNN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/CNN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00055, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2020_best.hdf5
44/44 - 8s - loss: 4.1106e-04 - R2: 0.9472 - mae: 0.0150 - val_loss: 5.5166e-04 - val_R2: 0.9292 - val_mae: 0.0172 - lr: 2.9505e-06 - 8s/epoch - 182ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00055 to 0.00055, saving model to D:/SGYL/SM_results_data/check_points/CNN\CNN_2020_best.hdf5
44/44 - 6s - loss: 4.1126e-04 - R2: 0.9472 - mae: 0.0150 - val_loss: 5.5139e-04 - val_R2: 0.9292 - val_mae: 0.0172 - lr: 2.9505e-06 - 6s/epoch - 144ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.00055
44/44 - 6s - loss: 4.1157e-04 - R2: 0.9471 - mae: 0.0150 - val_loss: 5.5183e-04 - val_R2: 0.9291 - val_mae: 0.0172 - lr: 2.9505e-06 - 6s/epoch - 139ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.00055
44/44 - 6s - loss: 4.1093e-04 - R2: 0.9472 - mae: 0.0150 - val_loss: 5.5159e-04 - val_R2: 0.9292 - val_mae: 0.0172 - lr: 2.9505e-06 - 

In [87]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/CNN/','CNN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/CNN/Best_Config/','CNN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2020
{'units_choice': 10, 'kernel_size': 5, 'dropout': False, 'activate_dense1': 'sigmoid', 'lr': 0.002950516092627498}
2850/2850 [==============================] - 13s 5ms/step - loss: 5.6503e-04 - R2: 0.9217 - mae: 0.0172


[0.0005650329985655844, 0.9216988682746887, 0.01724882982671261]

### aaa